In [12]:
#https://github.com/atinghosh/VAE-pytorch/blob/master/VAE_CNN_BCEloss.py
import os
import sys
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
from VAE_mnist_v1 import VAE_mnist
from VAE_CIFAR_v1 import VAE_CIFAR

In [13]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
CUDA = True
SEED = 1
BATCH_SIZE = 128
LOG_INTERVAL = 10
EPOCHS = 25
no_of_sample = 10
MNIST = False


ZDIMS = 20

torch.manual_seed(SEED)
if CUDA:
    torch.cuda.manual_seed(SEED)

In [14]:
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}

if MNIST:
    print("MNIST")
    train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist', train=True, download=True,transform=transforms.ToTensor()),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist', train=False, transform=transforms.ToTensor()),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)
else:
    print("CIFAR")
    train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./CIFAR10', train=True, download=True,transform=transforms.ToTensor()),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./CIFAR10', train=False, transform=transforms.ToTensor()),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    
# elif DATASET == 2:
#     print("CIFAR10_PROCESSED")
#     processed_CIFAR10_data = datasets.ImageFolder(root='cifar10_processed/', transform=transforms.ToTensor())
#     print(processed_CIFAR10_data.shape)
#     train_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./CIFAR10', train=True, download=True,transform=transforms.ToTensor()),
#         batch_size=BATCH_SIZE, shuffle=True, **kwargs)

#     test_loader = torch.utils.data.DataLoader(datasets.CIFAR10('./CIFAR10', train=False, transform=transforms.ToTensor()),
#         batch_size=BATCH_SIZE, shuffle=True, **kwargs)

CIFAR10_PROCESSED


AttributeError: 'ImageFolder' object has no attribute 'shape'

In [4]:
if MNIST:
    print("MNIST")
    model = VAE_mnist(ZDIMS, BATCH_SIZE, no_of_sample)
else:
    print("CIFAR10")
    model = VAE_CIFAR(ZDIMS, BATCH_SIZE, no_of_sample)
if CUDA:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

CIFAR10


In [5]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        if CUDA:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = model.loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.9f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [8]:
def test(epoch):
    model.eval()
    test_loss = 0

    # each data is of BATCH_SIZE (default 128) samples
    for i, (data, _) in enumerate(test_loader):
        if CUDA:
            # make sure this lives on the GPU
            data = data.cuda()

        # we're only going to infer, so no autograd at all required: volatile=True
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += model.loss_function(recon_batch, data, mu, logvar).item()
        if i == 0:
            n = min(data.size(0), 8)
            # for the first 128 batch of the epoch, show the first 8 input digits
            # with right below them the reconstructed output digits
            if MNIST:
                comparison = torch.cat([data[:n],
                                        recon_batch.view(BATCH_SIZE, 1, 28, 28)[:n]])
                save_image(comparison.data.cpu(),
                           './mnist/reconstruction_' + str(epoch) + '.png', nrow=n)
            else:
                comparison = torch.cat([data[:n],
                                        recon_batch.view(BATCH_SIZE, 3, 32, 32)[:n]])
                save_image(comparison.data.cpu(),
                           './CIFAR10/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test(epoch)

    # 64 sets of random ZDIMS-float vectors, i.e. 64 locations / MNIST
    # digits in latent space
    sample = Variable(torch.randn(64, ZDIMS))
    if CUDA:
        sample = sample.cuda()
    sample = model.decode(sample).cpu()

    # save out as an 8x8 matrix of MNIST`ii digits
    # this will give you a visual idea of how well latent space can generate things
    # that look like digits
    if MNIST:
        save_image(sample.data.view(64, 1, 28, 28),'./mnist/reconstruction' + str(epoch) + '.png')
    else:
        save_image(sample.data.view(64, 3, 32, 32),'./CIFAR10/reconstruction' + str(epoch) + '.png')

C:\Users\luke\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/50000 (0%)]	Loss: 0.004852
Train Epoch: 1 [1280/50000 (3%)]	Loss: 0.004786
Train Epoch: 1 [2560/50000 (5%)]	Loss: 0.004768
Train Epoch: 1 [3840/50000 (8%)]	Loss: 0.004908
Train Epoch: 1 [5120/50000 (10%)]	Loss: 0.004826
Train Epoch: 1 [6400/50000 (13%)]	Loss: 0.004888
Train Epoch: 1 [7680/50000 (15%)]	Loss: 0.004809
Train Epoch: 1 [8960/50000 (18%)]	Loss: 0.004820
Train Epoch: 1 [10240/50000 (20%)]	Loss: 0.004902
Train Epoch: 1 [11520/50000 (23%)]	Loss: 0.004822
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.004852
Train Epoch: 1 [14080/50000 (28%)]	Loss: 0.004798
Train Epoch: 1 [15360/50000 (31%)]	Loss: 0.004837
Train Epoch: 1 [16640/50000 (33%)]	Loss: 0.004831
Train Epoch: 1 [17920/50000 (36%)]	Loss: 0.004718
Train Epoch: 1 [19200/50000 (38%)]	Loss: 0.004871
Train Epoch: 1 [20480/50000 (41%)]	Loss: 0.004802
Train Epoch: 1 [21760/50000 (43%)]	Loss: 0.004848
Train Epoch: 1 [23040/50000 (46%)]	Loss: 0.004849
Train Epoch: 1 [24320/50000 (49%)]	Loss: 0.004864
Train Epoch: 1 

C:\Users\luke\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if sys.path[0] == '':


====> Test set loss: 0.0049
Train Epoch: 2 [0/50000 (0%)]	Loss: 0.004828
Train Epoch: 2 [1280/50000 (3%)]	Loss: 0.004921
Train Epoch: 2 [2560/50000 (5%)]	Loss: 0.004834
Train Epoch: 2 [3840/50000 (8%)]	Loss: 0.004794
Train Epoch: 2 [5120/50000 (10%)]	Loss: 0.004834
Train Epoch: 2 [6400/50000 (13%)]	Loss: 0.004906
Train Epoch: 2 [7680/50000 (15%)]	Loss: 0.004789
Train Epoch: 2 [8960/50000 (18%)]	Loss: 0.004787
Train Epoch: 2 [10240/50000 (20%)]	Loss: 0.004855
Train Epoch: 2 [11520/50000 (23%)]	Loss: 0.004805
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.004888
Train Epoch: 2 [14080/50000 (28%)]	Loss: 0.004878
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.004891
Train Epoch: 2 [16640/50000 (33%)]	Loss: 0.004876
Train Epoch: 2 [17920/50000 (36%)]	Loss: 0.004863
Train Epoch: 2 [19200/50000 (38%)]	Loss: 0.004837
Train Epoch: 2 [20480/50000 (41%)]	Loss: 0.004792
Train Epoch: 2 [21760/50000 (43%)]	Loss: 0.004885
Train Epoch: 2 [23040/50000 (46%)]	Loss: 0.004799
Train Epoch: 2 [24320/50000 (49%)]	Lo

====> Test set loss: 0.0048
Train Epoch: 6 [0/50000 (0%)]	Loss: 0.004857
Train Epoch: 6 [1280/50000 (3%)]	Loss: 0.004849
Train Epoch: 6 [2560/50000 (5%)]	Loss: 0.004861
Train Epoch: 6 [3840/50000 (8%)]	Loss: 0.004733
Train Epoch: 6 [5120/50000 (10%)]	Loss: 0.004805
Train Epoch: 6 [6400/50000 (13%)]	Loss: 0.004808
Train Epoch: 6 [7680/50000 (15%)]	Loss: 0.004744
Train Epoch: 6 [8960/50000 (18%)]	Loss: 0.004759
Train Epoch: 6 [10240/50000 (20%)]	Loss: 0.004823
Train Epoch: 6 [11520/50000 (23%)]	Loss: 0.004848
Train Epoch: 6 [12800/50000 (26%)]	Loss: 0.004824
Train Epoch: 6 [14080/50000 (28%)]	Loss: 0.004907
Train Epoch: 6 [15360/50000 (31%)]	Loss: 0.004828
Train Epoch: 6 [16640/50000 (33%)]	Loss: 0.004797
Train Epoch: 6 [17920/50000 (36%)]	Loss: 0.004891
Train Epoch: 6 [19200/50000 (38%)]	Loss: 0.004788
Train Epoch: 6 [20480/50000 (41%)]	Loss: 0.004828
Train Epoch: 6 [21760/50000 (43%)]	Loss: 0.004802
Train Epoch: 6 [23040/50000 (46%)]	Loss: 0.004846
Train Epoch: 6 [24320/50000 (49%)]	Lo

====> Test set loss: 0.0048
Train Epoch: 10 [0/50000 (0%)]	Loss: 0.004749
Train Epoch: 10 [1280/50000 (3%)]	Loss: 0.004742
Train Epoch: 10 [2560/50000 (5%)]	Loss: 0.004789
Train Epoch: 10 [3840/50000 (8%)]	Loss: 0.004879
Train Epoch: 10 [5120/50000 (10%)]	Loss: 0.004798
Train Epoch: 10 [6400/50000 (13%)]	Loss: 0.004782
Train Epoch: 10 [7680/50000 (15%)]	Loss: 0.004843
Train Epoch: 10 [8960/50000 (18%)]	Loss: 0.004779
Train Epoch: 10 [10240/50000 (20%)]	Loss: 0.004771
Train Epoch: 10 [11520/50000 (23%)]	Loss: 0.004874
Train Epoch: 10 [12800/50000 (26%)]	Loss: 0.004840
Train Epoch: 10 [14080/50000 (28%)]	Loss: 0.004678
Train Epoch: 10 [15360/50000 (31%)]	Loss: 0.004760
Train Epoch: 10 [16640/50000 (33%)]	Loss: 0.004738
Train Epoch: 10 [17920/50000 (36%)]	Loss: 0.004784
Train Epoch: 10 [19200/50000 (38%)]	Loss: 0.004803
Train Epoch: 10 [20480/50000 (41%)]	Loss: 0.004831
Train Epoch: 10 [21760/50000 (43%)]	Loss: 0.004719
Train Epoch: 10 [23040/50000 (46%)]	Loss: 0.004827
Train Epoch: 10 [2

Train Epoch: 13 [48640/50000 (97%)]	Loss: 0.004849
Train Epoch: 13 [31200/50000 (100%)]	Loss: 0.007511
====> Epoch: 13 Average loss: 0.004814775
====> Test set loss: 0.0048
Train Epoch: 14 [0/50000 (0%)]	Loss: 0.004876
Train Epoch: 14 [1280/50000 (3%)]	Loss: 0.004844
Train Epoch: 14 [2560/50000 (5%)]	Loss: 0.004883
Train Epoch: 14 [3840/50000 (8%)]	Loss: 0.004791
Train Epoch: 14 [5120/50000 (10%)]	Loss: 0.004814
Train Epoch: 14 [6400/50000 (13%)]	Loss: 0.004794
Train Epoch: 14 [7680/50000 (15%)]	Loss: 0.004700
Train Epoch: 14 [8960/50000 (18%)]	Loss: 0.004897
Train Epoch: 14 [10240/50000 (20%)]	Loss: 0.004737
Train Epoch: 14 [11520/50000 (23%)]	Loss: 0.004845
Train Epoch: 14 [12800/50000 (26%)]	Loss: 0.004814
Train Epoch: 14 [14080/50000 (28%)]	Loss: 0.004837
Train Epoch: 14 [15360/50000 (31%)]	Loss: 0.004865
Train Epoch: 14 [16640/50000 (33%)]	Loss: 0.004801
Train Epoch: 14 [17920/50000 (36%)]	Loss: 0.004788
Train Epoch: 14 [19200/50000 (38%)]	Loss: 0.004697
Train Epoch: 14 [20480/500

Train Epoch: 17 [44800/50000 (90%)]	Loss: 0.004831
Train Epoch: 17 [46080/50000 (92%)]	Loss: 0.004823
Train Epoch: 17 [47360/50000 (95%)]	Loss: 0.004723
Train Epoch: 17 [48640/50000 (97%)]	Loss: 0.004765
Train Epoch: 17 [31200/50000 (100%)]	Loss: 0.007641
====> Epoch: 17 Average loss: 0.004812304
====> Test set loss: 0.0048
Train Epoch: 18 [0/50000 (0%)]	Loss: 0.004787
Train Epoch: 18 [1280/50000 (3%)]	Loss: 0.004768
Train Epoch: 18 [2560/50000 (5%)]	Loss: 0.004848
Train Epoch: 18 [3840/50000 (8%)]	Loss: 0.004802
Train Epoch: 18 [5120/50000 (10%)]	Loss: 0.004827
Train Epoch: 18 [6400/50000 (13%)]	Loss: 0.004846
Train Epoch: 18 [7680/50000 (15%)]	Loss: 0.004839
Train Epoch: 18 [8960/50000 (18%)]	Loss: 0.004763
Train Epoch: 18 [10240/50000 (20%)]	Loss: 0.004809
Train Epoch: 18 [11520/50000 (23%)]	Loss: 0.004852
Train Epoch: 18 [12800/50000 (26%)]	Loss: 0.004882
Train Epoch: 18 [14080/50000 (28%)]	Loss: 0.004852
Train Epoch: 18 [15360/50000 (31%)]	Loss: 0.004746
Train Epoch: 18 [16640/500

Train Epoch: 21 [40960/50000 (82%)]	Loss: 0.004781
Train Epoch: 21 [42240/50000 (84%)]	Loss: 0.004793
Train Epoch: 21 [43520/50000 (87%)]	Loss: 0.004763
Train Epoch: 21 [44800/50000 (90%)]	Loss: 0.004851
Train Epoch: 21 [46080/50000 (92%)]	Loss: 0.004882
Train Epoch: 21 [47360/50000 (95%)]	Loss: 0.004789
Train Epoch: 21 [48640/50000 (97%)]	Loss: 0.004759
Train Epoch: 21 [31200/50000 (100%)]	Loss: 0.007596
====> Epoch: 21 Average loss: 0.004810778
====> Test set loss: 0.0048
Train Epoch: 22 [0/50000 (0%)]	Loss: 0.004766
Train Epoch: 22 [1280/50000 (3%)]	Loss: 0.004805
Train Epoch: 22 [2560/50000 (5%)]	Loss: 0.004821
Train Epoch: 22 [3840/50000 (8%)]	Loss: 0.004789
Train Epoch: 22 [5120/50000 (10%)]	Loss: 0.004764
Train Epoch: 22 [6400/50000 (13%)]	Loss: 0.004741
Train Epoch: 22 [7680/50000 (15%)]	Loss: 0.004805
Train Epoch: 22 [8960/50000 (18%)]	Loss: 0.004802
Train Epoch: 22 [10240/50000 (20%)]	Loss: 0.004795
Train Epoch: 22 [11520/50000 (23%)]	Loss: 0.004704
Train Epoch: 22 [12800/500

Train Epoch: 25 [37120/50000 (74%)]	Loss: 0.004792
Train Epoch: 25 [38400/50000 (77%)]	Loss: 0.004773
Train Epoch: 25 [39680/50000 (79%)]	Loss: 0.004831
Train Epoch: 25 [40960/50000 (82%)]	Loss: 0.004785
Train Epoch: 25 [42240/50000 (84%)]	Loss: 0.004812
Train Epoch: 25 [43520/50000 (87%)]	Loss: 0.004801
Train Epoch: 25 [44800/50000 (90%)]	Loss: 0.004843
Train Epoch: 25 [46080/50000 (92%)]	Loss: 0.004804
Train Epoch: 25 [47360/50000 (95%)]	Loss: 0.004739
Train Epoch: 25 [48640/50000 (97%)]	Loss: 0.004818
Train Epoch: 25 [31200/50000 (100%)]	Loss: 0.007536
====> Epoch: 25 Average loss: 0.004808904
====> Test set loss: 0.0048


In [ ]:
torch.save(model, "vae.pt")

In [ ]:
num_interpolation_points = 10
sample = Variable(torch.randn(2, ZDIMS))
first_point = sample[0]
last_point = sample[1]
interpolation_points_list = []
for i in np.linspace(0,1,num_interpolation_points):
    new_interpolation_point = (1-i)*first_point+i*last_point
    interpolation_points_list.append(new_interpolation_point)

interpolation_sample = Variable(torch.stack(interpolation_points_list))
if CUDA:
    interpolation_sample = interpolation_sample.cuda()
interpolation_sample = model.decode(interpolation_sample).cpu()
save_image(interpolation_sample.data.view(num_interpolation_points, 1, 28, 28),'./mnist/interpolation.png')